# 02_create_splits — Split único reproducible para todos los baselines

**Objetivo:** crear **una sola vez** los splits train/val estratificados y guardarlos en `data/splits/` para que **todos los baselines** (rule-based, TF-IDF, transformer) usen **exactamente los mismos ejemplos** y sean **comparables**.

**Outputs:**
- `data/splits/train_indices.csv` (índices para train)
- `data/splits/val_indices.csv` (índices para val)
- `data/splits/dataset_base.csv` (dataset normalizado base con índices)

**Reproducibilidad:** `random_state=42` fijo.

In [23]:
# ===============================================================
# Setup: Paths y configuración
# ===============================================================
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split

# Importar utilidades compartidas
try:
    from utils_shared import (
        setup_paths, 
        guess_text_col, 
        guess_label_col, 
        normalize_label,
        validate_file_exists
    )
    paths = setup_paths()
    DATA_PATH = paths['DATA_PATH']
    SPLITS_PATH = paths['SPLITS_PATH']
    print("[INFO] Usando utils_shared.py para configuración")
except ImportError:
    print("[WARNING] utils_shared.py no encontrado, usando configuración manual")
    BASE_PATH = Path.cwd()
    if BASE_PATH.name == "notebooks":
        BASE_PATH = BASE_PATH.parent
    DATA_PATH = BASE_PATH / "data"
    SPLITS_PATH = DATA_PATH / "splits"
    SPLITS_PATH.mkdir(exist_ok=True)
    
    # Definir funciones helper manualmente si no hay utils
    def guess_text_col(df):
        for c in ['texto', 'Motivo Consulta', 'text']:
            if c in df.columns: return c
        raise ValueError("No se encontró columna de texto")
    
    def guess_label_col(df):
        for c in ['etiqueta', 'Tipo', 'label']:
            if c in df.columns: return c
        return None
    
    def normalize_label(s):
        import unicodedata
        if pd.isna(s): return ""
        s = str(s).strip().lower()
        s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
        return {'depresivo': 'depresion'}.get(s, s)

# ---------------------------------------------------------------
# Detectar archivo de entrada (priorizar ips_clean.csv)
# ---------------------------------------------------------------
INPUT_FILE = DATA_PATH / 'ips_clean.csv'
if not INPUT_FILE.exists():
    INPUT_FILE = DATA_PATH / 'ips_raw.csv'
    print("[WARNING] No se encontró ips_clean.csv, usando ips_raw.csv")
    print("          Recomendación: ejecutar 01_eda_understanding.ipynb primero")
    if not INPUT_FILE.exists():
        raise FileNotFoundError(
            f"[ERROR] No se encontró ni ips_clean.csv ni ips_raw.csv en {DATA_PATH}\n"
            f"        Ejecuta 01_eda_understanding.ipynb para generar ips_clean.csv"
        )

print(f"[INFO] INPUT_FILE: {INPUT_FILE}")
print(f"[INFO] SPLITS_PATH: {SPLITS_PATH}")

[INFO] Usando utils_shared.py para configuración
[INFO] INPUT_FILE: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/ips_clean.csv
[INFO] SPLITS_PATH: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits


## 1) Carga y normalización base (sin preprocesamiento de texto)

In [24]:
# ===============================================================
# Carga y normalización del dataset base
# ===============================================================
# Cargar dataset (ya debería estar limpio si viene de ips_clean.csv)
df_raw = pd.read_csv(INPUT_FILE)
print(f"[INFO] Dataset cargado: {len(df_raw)} filas")

# Detectar columnas automáticamente usando utils
text_col = guess_text_col(df_raw)
label_col = guess_label_col(df_raw)

if label_col is None:
    raise ValueError(
        "[ERROR] No se encontró columna de etiquetas en el dataset.\n"
        f"        Columnas disponibles: {list(df_raw.columns)}"
    )

print(f"[INFO] Columnas detectadas:")
print(f"  Texto: '{text_col}'")
print(f"  Etiqueta: '{label_col}'")

# ---------------------------------------------------------------
# Filtrar y normalizar etiquetas
# ---------------------------------------------------------------
# Eliminar filas sin texto o etiqueta
df = df_raw.dropna(subset=[text_col, label_col]).copy()
print(f"  Después de remover nulos: {len(df)} filas")

# Normalizar etiquetas (ansiedad/depresion)
df[label_col] = df[label_col].map(normalize_label)

# Filtrar solo ansiedad y depresión (clasificación binaria)
df = df[df[label_col].isin(['ansiedad','depresion'])].copy()
print(f"  Después de filtrar A/D: {len(df)} filas")

# ---------------------------------------------------------------
# Eliminar duplicados (por si viene de ips_raw.csv)
# ---------------------------------------------------------------
n_before = len(df)
df = df.drop_duplicates(subset=[text_col]).copy()
n_after = len(df)

if n_before > n_after:
    print(f"[WARNING] Eliminados {n_before - n_after} duplicados")
    print(f"          Recomendación: Usar ips_clean.csv para evitar esto en el futuro")

# ---------------------------------------------------------------
# Resetear índice y crear row_id único
# ---------------------------------------------------------------
df = df.reset_index(drop=True)
df['row_id'] = df.index

print(f"\n[INFO] Dataset final preparado: {len(df)} ejemplos únicos")
print(f"\nDistribución de clases:")
print(df[label_col].value_counts())

[INFO] Dataset cargado: 3127 filas
[INFO] Columnas detectadas:
  Texto: 'texto'
  Etiqueta: 'etiqueta'
  Después de remover nulos: 3126 filas
  Después de filtrar A/D: 3126 filas

[INFO] Dataset final preparado: 3126 ejemplos únicos

Distribución de clases:
etiqueta
depresion    2201
ansiedad      925
Name: count, dtype: int64


## 2) Split estratificado (80/20) con semilla fija

In [25]:
# ===============================================================
# Estrategia de Split: 80/20 estratificado con semilla fija
# ===============================================================
# 
# ¿Por qué estos parámetros?
# 
# 1) **RANDOM_STATE = 42**: 
#    - Fija la semilla aleatoria para reproducibilidad total
#    - Todos los baselines verán exactamente los mismos ejemplos
#    - Permite comparaciones justas entre modelos
#
# 2) **TEST_SIZE = 0.2** (80/20):
#    - Estándar de la industria para datasets pequeños/medianos
#    - 80% train proporciona suficientes ejemplos para aprender patrones
#    - 20% val proporciona validación confiable sin desperdiciar datos
#
# 3) **stratify=df[label_col]**:
#    - Mantiene la proporción de clases en train y val
#    - Evita desbalances accidentales (ej: val con 90% ansiedad)
#    - Crítico para evaluar correctamente modelos en datos desbalanceados
#
# ===============================================================

RANDOM_STATE = 42
TEST_SIZE = 0.2

train_idx, val_idx = train_test_split(
    df.index,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=df[label_col]
)

# Reportar resultados del split
print(f"[INFO] Split creado exitosamente:")
print(f"  Train: {len(train_idx)} ejemplos ({len(train_idx)/len(df)*100:.1f}%)")
print(f"  Val:   {len(val_idx)} ejemplos ({len(val_idx)/len(df)*100:.1f}%)")
print(f"  Total: {len(df)} ejemplos")

print(f"\nDistribución en Train:")
train_dist = df.loc[train_idx, label_col].value_counts()
for label, count in train_dist.items():
    print(f"  {label}: {count} ({count/len(train_idx)*100:.1f}%)")

print(f"\nDistribución en Val:")
val_dist = df.loc[val_idx, label_col].value_counts()
for label, count in val_dist.items():
    print(f"  {label}: {count} ({count/len(val_idx)*100:.1f}%)")

# Validación: verificar que estratificación funcionó correctamente
train_prop = train_dist / len(train_idx)
val_prop = val_dist / len(val_idx)
max_diff = (train_prop - val_prop).abs().max()

if max_diff < 0.05:  # Diferencia < 5%
    print(f"\n[INFO] Estratificación exitosa (diferencia máxima: {max_diff*100:.2f}%)")
else:
    print(f"\n[WARNING] Estratificación con diferencia de {max_diff*100:.2f}% (verificar)")

[INFO] Split creado exitosamente:
  Train: 2500 ejemplos (80.0%)
  Val:   626 ejemplos (20.0%)
  Total: 3126 ejemplos

Distribución en Train:
  depresion: 1760 (70.4%)
  ansiedad: 740 (29.6%)

Distribución en Val:
  depresion: 441 (70.4%)
  ansiedad: 185 (29.6%)

[INFO] Estratificación exitosa (diferencia máxima: 0.05%)


## 3) Guardar splits y dataset base

In [26]:
# ===============================================================
# Guardar splits unificados para todos los baselines
# ===============================================================
#
# Estructura de archivos generada:
#
# 1) **train_indices.csv / val_indices.csv**:
#    - Contienen únicamente los row_id de cada split
#    - Permite a cada baseline cargar exactamente los mismos ejemplos
#    - Ligero (solo IDs), no duplica datos de texto
#
# 2) **dataset_base.csv**:
#    - Contiene: row_id + texto original + etiqueta normalizada
#    - Es el "dataset maestro" que usan TODOS los baselines
#    - Cada baseline carga este archivo y filtra con los índices
#
# ¿Por qué esta estrategia?
# - **Reproducibilidad**: Un único punto de verdad para los datos
# - **Comparabilidad**: Todos los modelos ven exactamente lo mismo
# - **Flexibilidad**: Cada baseline puede preprocesar el texto a su manera
#   (ej: rule-based no lo limpia, TF-IDF sí lo limpia)
# - **Mantenibilidad**: Cambios en splits se propagan automáticamente
#
# ===============================================================

# Guardar índices de train y val
train_indices_path = SPLITS_PATH / 'train_indices.csv'
val_indices_path = SPLITS_PATH / 'val_indices.csv'

pd.DataFrame({'row_id': train_idx}).to_csv(train_indices_path, index=False)
pd.DataFrame({'row_id': val_idx}).to_csv(val_indices_path, index=False)

# Guardar dataset base (con texto original y etiqueta normalizada)
# IMPORTANTE: No aplicamos limpieza aquí, cada baseline decide su estrategia
df_base = df[['row_id', text_col, label_col]].copy()
dataset_base_path = SPLITS_PATH / 'dataset_base.csv'
df_base.to_csv(dataset_base_path, index=False, encoding='utf-8')

# Reportar archivos creados
print("\n" + "="*60)
print("[INFO] Archivos de splits creados exitosamente")
print("="*60)
print(f"\nUbicación: {SPLITS_PATH}/")
print(f"\n1. train_indices.csv")
print(f"   - {len(train_idx)} índices de entrenamiento")
print(f"\n2. val_indices.csv")
print(f"   - {len(val_idx)} índices de validación")
print(f"\n3. dataset_base.csv")
print(f"   - {len(df_base)} ejemplos con texto original")
print(f"   - Columnas: {list(df_base.columns)}")
print(f"   - Origen: {INPUT_FILE.name}")

print(f"\n" + "="*60)
print("IMPORTANTE: Todos los baselines DEBEN usar estos archivos")
print("="*60)
print(f"Los baselines deben:")
print(f"  1. Cargar dataset_base.csv")
print(f"  2. Cargar train_indices.csv y val_indices.csv")
print(f"  3. Filtrar ejemplos usando row_id")
print(f"  4. Aplicar su propia estrategia de preprocesamiento")
print(f"\nVer notebooks 02_baseline_*.ipynb para ejemplos de uso")


[INFO] Archivos de splits creados exitosamente

Ubicación: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits/

1. train_indices.csv
   - 2500 índices de entrenamiento

2. val_indices.csv
   - 626 índices de validación

3. dataset_base.csv
   - 3126 ejemplos con texto original
   - Columnas: ['row_id', 'texto', 'etiqueta']
   - Origen: ips_clean.csv

IMPORTANTE: Todos los baselines DEBEN usar estos archivos
Los baselines deben:
  1. Cargar dataset_base.csv
  2. Cargar train_indices.csv y val_indices.csv
  3. Filtrar ejemplos usando row_id
  4. Aplicar su propia estrategia de preprocesamiento

Ver notebooks 02_baseline_*.ipynb para ejemplos de uso
